# Data Splits Truncated
Here, we split in the same way as in `data_splits.ipynb`, but then we restrict the number of points in the training data.
There are two ways we could do this:
1. Take the original splits and drop random examples. Problem: As we go to smaller training sets, this will lead to us inadvertently removing all the examples of one compound from the training data but not from validation/test.
2. We do (procedurally) the same splits as before, but with different train-test ratios.

Option 2 is safer to do w.r.t. retaining the split dimensionality.

### 0D Split
For the 0D split, we use a random train-test split.
Standard: We use a 80/10/10 split into train, val, and test set.

Truncated: 
- 40/30/30 -> 16k training samples
- 20/40/40 -> 8k training samples
- 10/45/45 -> 4k training samples
- 5/50/45 -> 2k training samples
- 2.5/50/47.5 -> 1k training samples
- 1.25/50/48.75 -> 500 training samples
- 0.625/50/49.375 -> 250 training samples

(numbers of training samples are averaged over folds and approximate)

### 1D Split
For the 1D split, we use a (1D) GroupShuffleSplit.
Standard: As groups, we use either initiator, monomer, or terminator (3x3 splits).

Truncated:
(for uniform results, we pick only one dimension to split on: Initiators)
- 80/10/10 -> 31,250 training samples (53 I) (this is not equivalent to the "standard" 1D split b/c here we split on initiators 9 times)
- 40/30/30 -> 15,600 training samples (26 I)
- 20/40/40 -> 8,100 training samples (13 I)
- 10/45/45 -> 3,680 training samples (6 I)
- 5/50/45 -> 1,668 training samples (3 I)
- 2.5/50/47.5 -> 487 training samples (1 I)
- 1.25/50/48.75 -> not possible b/c training set will be empty on some folds

(numbers of training samples are averaged over folds)

### 2D split
For the 2D split, we use a (2D) GroupShuffleSplit.
Standard: As groups, we use either [initiator, monomer], [monomer, terminator] or [initiator, terminator]. (3x3)

Truncated:
(for uniform results, we pick only one set of two dimensions to split on: Initiators & Monomers)
- 80/10/10 -> 24,562 training samples (53 I, 56 M)
- 60/20/20 -> 13,802 training samples (39 I, 41.7 M)
- 40/30/30 -> 6,046 training samples (26 I, 27.9 M)
- 30/35/35 -> 3,193 training samples (19 I, 20.6 M)
- 20/40/40 -> 1,475 training samples (13 I, 13.7 M)
- 15/45/40 -> 812 training samples (10 I, 10 M)
- 10/45/45 -> 355 training samples (6 I, 6.7 M)
- 7.5/47.5/45 -> 163 training samples (4 I, 4.9 M)
- 5/50/45 -> 78 training samples (3 I, 2.9 M)
- 2/(all-2)/48 -> 38 training samples (2 I, 1.8 M) (here we define the train/val split such that 2 groups are in train)

(numbers of training samples are averaged over folds. In total there are 67 I & 72 M)

### 3D split
For the 3D split, we use a (3D) GroupShuffleSplit.

Truncated:
- 80/10/10 -> 19,725 training samples (53 I, 55.8 M, 32 T) n.b. this has extremely few val/test samples
- 70/15/15 -> 12,975 training samples (46 I, 49.4 M, 28 T)
- 60/20/20 -> 7,948 training samples (39 I, 41.4 M, 24 T) (this should be close to the standard split)
- 50/25/25 -> 4,747 training samples (33 I, 35.5 M, 20 T)
- 40/30/30 -> 2,399 training samples (26 I, 27.8 M, 16 T)
- 34/33/33 -> 1,313 training samples (22 I, 23.3 M, 13 T)
- 30/35/35 -> 901 training samples (19 I, 20.3 M, 12 T)
- 25/40/35 -> 529 training samples (16 I, 16.9 M, 10 T)
- 20/40/40 -> 273 training samples (13 I, 13.3 M, 8 T)
- 15/45/40 -> 112 training samples (10 I, 9.9 M, 6 T)
- 10/45/45 -> 32 training samples (5.8 I, 6 M, 4 T)

## Diasteromers
For the 0D split, this is not important, but for 1D/2D/3D, we want to define the groups such that diastereomers will always be in the same group. These splits will receive a `_dia` suffix

## Synthetic data
For the 1D/2D/3D problems we use a synthetically amended data set. Splits of the synthetically ammended data set will receive a `_synthetic` suffix.

In [ ]:
import pathlib
import sys

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, ShuffleSplit

from src.definitions import DATA_DIR
from src.util.train_test_split import GroupShuffleSplitND
from util import write_indices_and_stats

In [ ]:
# Load data
data_filename = "synferm_dataset_2023-09-05_40018records.csv"
data_name = data_filename.rsplit("_", maxsplit=1)[0]
df = pd.read_csv(DATA_DIR / "curated_data" / data_filename)
df.shape

In [ ]:
df.head()

## 0D split

In [ ]:
splitter = ShuffleSplit(n_splits=9, test_size=0.3, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.3/0.7, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, sizes, pos_class, split_dimension=0, save_indices=True, train_size=40, total_size=len(df), data_name=data_name
)

In [ ]:
splitter = ShuffleSplit(n_splits=9, test_size=0.4, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.4/0.6, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, sizes, pos_class, split_dimension=0, save_indices=True, train_size=20, total_size=len(df), data_name=data_name
)

In [ ]:
splitter = ShuffleSplit(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.45/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, sizes, pos_class, split_dimension=0, save_indices=True, train_size=10, total_size=len(df), data_name=data_name
)

In [ ]:
splitter = ShuffleSplit(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.5/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, sizes, pos_class, split_dimension=0, save_indices=True, train_size=5, total_size=len(df), data_name=data_name
)

In [ ]:
splitter = ShuffleSplit(n_splits=9, test_size=0.475, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.5/0.525, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, sizes, pos_class, split_dimension=0, save_indices=True, train_size=2.5, total_size=len(df), data_name=data_name
)

In [ ]:
splitter = ShuffleSplit(n_splits=9, test_size=0.4875, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.5/0.5125, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, sizes, pos_class, split_dimension=0, save_indices=True, train_size=1.25, total_size=len(df), data_name=data_name
)

In [ ]:
splitter = ShuffleSplit(n_splits=9, test_size=0.49375, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.5/0.50625, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, sizes, pos_class, split_dimension=0, save_indices=True, train_size=0.625, total_size=len(df), data_name=data_name
)

## 1D split

In [ ]:
def split_1d(splitter, inner_splitter):
    indices = []
    sizes = []
    pos_class = []
    unique_initiators = []
    unique_monomers = []
    unique_terminators = []
    for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
        # inner split
        train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
        # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
        idx_train = idx_train_val[train]
        idx_val = idx_train_val[val]
        # add to list
        indices.append((idx_train, idx_val, idx_test))
        sizes.append((len(idx_train), len(idx_val), len(idx_test)))
        pos_class.append(
            (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
             np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
             np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
            )
        )
        unique_initiators.append((len(df['I_long'][idx_train].drop_duplicates()), len(df['I_long'][idx_val].drop_duplicates()), len(df['I_long'][idx_test].drop_duplicates())))
        unique_monomers.append((len(df['M_long'][idx_train].drop_duplicates()), len(df['M_long'][idx_val].drop_duplicates()), len(df['M_long'][idx_test].drop_duplicates())))
        unique_terminators.append((len(df['T_long'][idx_train].drop_duplicates()), len(df['T_long'][idx_val].drop_duplicates()), len(df['T_long'][idx_test].drop_duplicates())))
    
    return indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators


In [ ]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.1, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.1/0.9, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_1d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=80
)

In [ ]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.3, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.3/0.7, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_1d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=40
)

In [ ]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.4, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.4/0.6, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_1d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=20
)

In [ ]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.45/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_1d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=10
)

In [ ]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.5/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_1d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=5
)

In [ ]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.475, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.5/0.525, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_1d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=2.5
)

## 2D split

In [ ]:
def split_2d(splitter, inner_splitter):
    indices = []
    sizes = []
    pos_class = []
    unique_initiators = []
    unique_monomers = []
    unique_terminators = []
    for idx_train_val, idx_test in splitter.split(df, groups=df[["I_long", "M_long"]]):
        train, val = next(inner_splitter.split(df.iloc[idx_train_val], groups=df[["I_long", "M_long"]].iloc[idx_train_val]))
        # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
        idx_train = idx_train_val[train]
        idx_val = idx_train_val[val]
        indices.append((idx_train, idx_val, idx_test))
        sizes.append((len(idx_train), len(idx_val), len(idx_test)))
        pos_class.append(
            (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
             np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
             np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
            )
        )
        unique_initiators.append((len(df['I_long'][idx_train].drop_duplicates()), len(df['I_long'][idx_val].drop_duplicates()), len(df['I_long'][idx_test].drop_duplicates())))
        unique_monomers.append((len(df['M_long'][idx_train].drop_duplicates()), len(df['M_long'][idx_val].drop_duplicates()), len(df['M_long'][idx_test].drop_duplicates())))
        unique_terminators.append((len(df['T_long'][idx_train].drop_duplicates()), len(df['T_long'][idx_val].drop_duplicates()), len(df['T_long'][idx_test].drop_duplicates())))
    
    return indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators


In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.1, random_state=42)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.1/0.9, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_2d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=80
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.2, random_state=42)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.2/0.8, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_2d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=60
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.3, random_state=42)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.3/0.7, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_2d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=40
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.35, random_state=42)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.35/0.65, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_2d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=30
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.4, random_state=42)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.4/0.6, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_2d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=20
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.4, random_state=42)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.45/0.6, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_2d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=15
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.45/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_2d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=10
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.475/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_2d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=7.5
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.5/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_2d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=5
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.48, random_state=42)
inner_splitter = GroupShuffleSplitND(n_splits=1, train_size=2, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_2d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=2
)

## 3D split

In [ ]:
def split_3d(splitter, inner_splitter):
    indices = []
    sizes = []
    pos_class = []
    unique_initiators = []
    unique_monomers = []
    unique_terminators = []
    for idx_train_val, idx_test in splitter.split(df, groups=df[["I_long", "M_long", "T_long"]]):
        train, val = next(inner_splitter.split(df.iloc[idx_train_val], groups=df[["I_long", "M_long", "T_long"]].iloc[idx_train_val]))
        # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
        idx_train = idx_train_val[train]
        idx_val = idx_train_val[val]
        indices.append((idx_train, idx_val, idx_test))
        sizes.append((len(idx_train), len(idx_val), len(idx_test)))
        pos_class.append(
            (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
             np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
             np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
            )
        )
        unique_initiators.append((len(df['I_long'][idx_train].drop_duplicates()), len(df['I_long'][idx_val].drop_duplicates()), len(df['I_long'][idx_test].drop_duplicates())))
        unique_monomers.append((len(df['M_long'][idx_train].drop_duplicates()), len(df['M_long'][idx_val].drop_duplicates()), len(df['M_long'][idx_test].drop_duplicates())))
        unique_terminators.append((len(df['T_long'][idx_train].drop_duplicates()), len(df['T_long'][idx_val].drop_duplicates()), len(df['T_long'][idx_test].drop_duplicates())))

    return indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators


In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.1, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.1/0.9, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=80
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.15, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.15/0.85, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=70
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.2, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.2/0.8, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=60
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.25, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.25/0.75, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=50
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.3, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.3/0.7, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=40
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.33, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.33/0.67, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=34
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.35, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.35/0.65, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=30
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.35, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.40/0.65, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=25
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.40, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.40/0.60, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=20
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.40, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.45/0.60, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=15
)

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.45, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.45/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices, sizes, pos_class, unique_initiators, unique_monomers, unique_terminators = split_3d(splitter, inner_splitter)

print(sizes)
print(pos_class)
print(unique_initiators)
print(unique_monomers)
print(unique_terminators)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    n_initiators=unique_initiators, 
    n_monomers=unique_monomers, 
    n_terminators=unique_terminators, 
    train_size=10
)